In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np

In [ ]:
# !pip install transformers==2.1.0
!pip install gensim==4.1.2
!pip3 install --user underthesea
!pip install tensorflow_addons
# !pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#from pyvi import ViTokenizer, ViPosTagger
import regex as re
from underthesea import word_tokenize
import numpy as np
import pandas as pd
import gensim
import sklearn
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
#from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from tensorflow.keras.layers import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import Adam
# Chia tập train/test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

In [ ]:
def text_preprocess(document):
    # xóa html code
    document = remove_html(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    # tách từ
    document = word_tokenize(document, format="text")
    return document

In [ ]:
# lst_stopword = []
# with open('/content/drive/MyDrive/KhanhMD/Data/vietnamese-stopwords.txt','r') as f:
#     for line in f:
#         for word in line.split():
#            lst_stopword.append(word)  
    
# def remove_stopwords(line):
#     words = []
#     for word in line.strip().split():
#         if word not in lst_stopword:
#             words.append(word)
#     return ' '.join(words)

In [ ]:
with open('/content/drive/MyDrive/KhanhMD/Data/data_pos.txt', 'r') as file:
    data_pos = file.read().replace('\n', '/27121998/')

In [ ]:
with open('/content/drive/MyDrive/KhanhMD/Data/data_neg.txt', 'r') as file:
    data_neg = file.read().replace('\n', '/27121998/')

In [ ]:
clean_data_pos = text_preprocess(data_pos)

In [ ]:
clean_data_neg = text_preprocess(data_neg)

In [ ]:
data_pos[:1000]

'Đây là nhà_thờ được cho là lớn nhất Huế , lại nằm gần_như ở trung_tâm thành_phố . Với lối kiến_trúc của người Pháp , nhà_thờ mang nét cổ_kính pha lẫn với hiện_đại rất châu Âu nên bản_thân mình thấy kiến_trúc nhà_thờ rất đẹp , toát lên cái cao_quý của nhà_thờ . Không_gian ở đây khá yên_bình . Và mỗi dịp giáng_sinh về , nơi đây luôn là nơi rực_rỡ và đẹp nhất .   /27121998/Cơm hến , bún hến ở chợ Phú lộc nghe lâu rồi mà mới có dịp được ăn . Trước giờ toàn ăn cơm hến , bún hến ở đường Hải_Phòng thôi . Có thời_gian lên ăn_ở Chợ_Phú_Lộc , hến béo hơn , thơm hơn nhiều . Công_nhận nước_ngọt nữa . Gía chỉ 10 / tô quá rẻ rồi . /27121998/Resort ngay trung_tâm , cách phố cổ tầm 700m-1km và bạn dễ_dàng đi bộ hoặc đạp xe vào phố . Phòng_ốc xinh_xắn , đặc_biệt là hồ bơi ^ ^ . Hồ bơi không chứa clo mà thay bằng nước_biển lọc mặn . Spa được cộng trong tiền phòng nên mỗi đêm ở bạn được cộng 1 buổi spa luôn nhé /27121998/Chỉ cách phổ cổ khoảng 1km nên khách_sạn này rất tiện_lợi . Do mới xây nên rất mới 

In [ ]:
clean_data_neg[:1000]

'mua có mỗi bingsu thập_cẩm 45 k mà mình f đợi hơn 20 hỏi lại thì nv tl có r nhg bảo chờ thêm 15 nữa tụi e lm liền mình k biết có ngon k nhg cũng muốn ăn thử thiết_nghĩ nv quán nên xem_lại cách pv và nc vs khách 27121998 quán này khá là nổi_tiếng nay mới có dịp ghé thử năm ở vii trí khá dễ tìm quán hơi nhỏ đi chiều_tối là đông lắm phải đợi lâu nhân_viên thì bình_thường à vì do đông quá nên cũng thông_cảm món ăn mình gọi gà sốt phô_mai cay gà quá bở thịt ăn vào k thơm món này đã có mỡ rồi cộng thêm da_gà nữa ăn rất ngáy cảm_giác ngán lắm cơm trộn kim_chi hq thì_phải ăn hơi cay nóng_hổi luôn vị ăn ok nhưng k đặc_sắc với giá đó thì ăn k đáng tiền khoai_tây chiên dở và ăn vị k ngon tóm_lại k như kì_vọng của mình giá_cả ở đây quá đắt so với chất_lượng mà đi ăn phải chầu_chực như_thế thì cũng hơi mệt à có nước_lọc miễn_phí nha mọi người tới ăn nhớ hỏi 27121998 thứ 6 nào ta cùng quẩy vuvuzela beer club chung hệ_thống gogi_house daruma kichi kichi nên giá cũng chưa 10 vat thanh_toán trước 20 g

In [ ]:
text_pos = clean_data_pos.split(' 27121998 ')

In [ ]:
text_pos[:10]

['đây là nhà_thờ được cho là lớn nhất huế lại nằm gần_như ở trung_tâm thành_phố với lối kiến_trúc của người pháp nhà_thờ mang nét cổ_kính pha lẫn với hiện_đại rất châu_âu nên bản_thân mình thấy kiến_trúc nhà_thờ rất đẹp toát lên cái cao_quý của nhà_thờ không_gian ở đây khá yên_bình và mỗi dịp giáng_sinh về nơi đây luôn là nơi rực_rỡ và đẹp nhất',
 'cơm hến bún hến ở chợ phú lộc nghe lâu rồi mà mới có dịp được ăn trước giờ toàn ăn cơm hến bún hến ở đường hải_phòng thôi có thời_gian lên ăn_ở chợ_phú_lộc hến béo hơn thơm hơn nhiều công_nhận nước_ngọt nữa gía chỉ 10 tô quá rẻ rồi',
 'resort ngay trung_tâm cách phố cổ tầm 700 m 1 km và bạn dễ_dàng đi bộ hoặc đạp xe vào phố phòng_ốc xinh_xắn đặc_biệt là hồ bơi hồ bơi không chứa clo mà thay bằng nước_biển lọc mặn spa được cộng trong tiền_phòng nên mỗi đêm ở bạn được cộng 1 buổi spa luôn nhé',
 'chỉ cách phổ cổ khoảng 1 km nên khách_sạn này rất tiện_lợi do mới xây nên rất mới đẹp phòng_ốc có loại phòng nhỏ và phòng bự có bồ sục mình ở loại phò

In [ ]:
text_neg = clean_data_neg.split(' 27121998 ')
text_neg[:10]

['mua có mỗi bingsu thập_cẩm 45 k mà mình f đợi hơn 20 hỏi lại thì nv tl có r nhg bảo chờ thêm 15 nữa tụi e lm liền mình k biết có ngon k nhg cũng muốn ăn thử thiết_nghĩ nv quán nên xem_lại cách pv và nc vs khách',
 'quán này khá là nổi_tiếng nay mới có dịp ghé thử năm ở vii trí khá dễ tìm quán hơi nhỏ đi chiều_tối là đông lắm phải đợi lâu nhân_viên thì bình_thường à vì do đông quá nên cũng thông_cảm món ăn mình gọi gà sốt phô_mai cay gà quá bở thịt ăn vào k thơm món này đã có mỡ rồi cộng thêm da_gà nữa ăn rất ngáy cảm_giác ngán lắm cơm trộn kim_chi hq thì_phải ăn hơi cay nóng_hổi luôn vị ăn ok nhưng k đặc_sắc với giá đó thì ăn k đáng tiền khoai_tây chiên dở và ăn vị k ngon tóm_lại k như kì_vọng của mình giá_cả ở đây quá đắt so với chất_lượng mà đi ăn phải chầu_chực như_thế thì cũng hơi mệt à có nước_lọc miễn_phí nha mọi người tới ăn nhớ hỏi',
 'thứ 6 nào ta cùng quẩy vuvuzela beer club chung hệ_thống gogi_house daruma kichi kichi nên giá cũng chưa 10 vat thanh_toán trước 20 g hằng ngà

In [ ]:
import pandas as pd
df = pd.DataFrame(text_pos, columns=["text"])
df.to_csv('text_pos.csv', index=False)

In [ ]:
text_pos_pd = pd.read_csv('text_pos.csv')
text_pos_pd[:10]

,text
0,đây là nhà_thờ được cho là lớn nhất huế lại nằ...
1,cơm hến bún hến ở chợ phú lộc nghe lâu rồi mà ...
2,resort ngay trung_tâm cách phố cổ tầm 700 m 1 ...
3,chỉ cách phổ cổ khoảng 1 km nên khách_sạn này ...
4,hôm_nay mình quay lại quán cùng_với mấy người ...
5,bánh và nước rất rất ngon phục_vụ nhiệt_tình c...
6,hôm đấy ngẫu_hứng tới nồi vì đang có chương_tr...
7,nghe tên quán_từ rất lâu rồi đọc review cũng t...
8,mình đến nồi_bay hôm trước và có gọi bánh mous...
9,tổng_hợp từ 2 lần em đến với nồi_bay bình_thườ...


In [ ]:
text_pos_pd=text_pos_pd.assign(label=lambda x: 1)

In [ ]:
df_0 = pd.DataFrame(text_neg, columns=["text"])
df_0.to_csv('text_neg.csv', index=False)

In [ ]:
text_neg_pd = pd.read_csv('text_neg.csv')
text_neg_pd[:10]

,text
0,mua có mỗi bingsu thập_cẩm 45 k mà mình f đợi ...
1,quán này khá là nổi_tiếng nay mới có dịp ghé t...
2,thứ 6 nào ta cùng quẩy vuvuzela beer club chun...
3,mình đi với nhóm tổng_cộng 4 người ăn chỉ có k...
4,nhân_viên phục_vụ không mấy tận_tình đồ_ăn ra ...
5,vào đây thì hết bàn nhưng mình vẫn ngồi đợi bì...
6,hôm_nay đến aeon có ghé vào quán món ăn cũng k...
7,mình vào lúc 8 h tối cũng không đông khách lắm...
8,thức_ăn quá_đỗi bình_thường miếng xôi của bạn ...
9,lần đầu_tiên ăn_chắc cũng là lần cuối ăn_ở đây...


In [ ]:
text_neg_pd=text_neg_pd.assign(label=lambda x: 0)

In [ ]:
df_concat_col = pd.concat([text_pos_pd, text_neg_pd])
df_concat_col.to_csv("data.csv", index=False)

In [ ]:
c = pd.read_csv("data.csv")
c[:10]

,text,label
0,đây là nhà_thờ được cho là lớn nhất huế lại nằ...,1
1,cơm hến bún hến ở chợ phú lộc nghe lâu rồi mà ...,1
2,resort ngay trung_tâm cách phố cổ tầm 700 m 1 ...,1
3,chỉ cách phổ cổ khoảng 1 km nên khách_sạn này ...,1
4,hôm_nay mình quay lại quán cùng_với mấy người ...,1
5,bánh và nước rất rất ngon phục_vụ nhiệt_tình c...,1
6,hôm đấy ngẫu_hứng tới nồi vì đang có chương_tr...,1
7,nghe tên quán_từ rất lâu rồi đọc review cũng t...,1
8,mình đến nồi_bay hôm trước và có gọi bánh mous...,1
9,tổng_hợp từ 2 lần em đến với nồi_bay bình_thườ...,1


In [ ]:
c = c.sample(frac=1).reset_index(drop=True)

In [ ]:
df = c.copy()
df[:50]

,text,label
0,quá sợ ăn cái món nè ăn chẳng ra j ăn dk 30 p ...,0
1,qua ́ n vơ ́ i phong ca ́ ch ba ̀ y_tri_́ kha ...,0
2,đi qua quán này nhiều lần nhưng k vào vì quán ...,0
3,cafe_ngon_và_không gian thoải_mái có cả kịch c...,1
4,bánh_mì chảo pate trứng pate nhuyễn mềm không ...,0
5,thức_ăn đồ_uống cũng được giá_cả ổn nghe nhiều...,0
6,ăn rồi mới thấy nó bình_thường mà có gì ngon đ...,0
7,cuối tuần vừa_rồi mình có ghé đây chơi để tận_...,0
8,ở đây phục_vụ quá kém dù giá_cả có_vẻ hợp_lý n...,0
9,mình thường uống trà sữa lắm phải nói là bị gh...,0


In [ ]:
df.shape

(24734, 2)

In [ ]:
text = clean_data_pos +' '+ clean_data_neg

In [ ]:
text = text.split(' 27121998 ')

In [ ]:
split_text = [s.split() for s in text]

In [ ]:
len(split_text)

24734

In [ ]:
w2v = gensim.models.Word2Vec(split_text,
                                   vector_size=100,
                                   window=5,
                                   min_count=1)
words = set(w2v.wv.index_to_key)

In [ ]:
w2v.train(split_text ,epochs=10,total_examples=len(split_text))

(17854615, 21409250)

In [ ]:
vocab=w2v.wv.key_to_index
print("The total number of words are : ",len(vocab))

The total number of words are :  35482


In [ ]:
vocab=list(vocab.keys())

In [ ]:
word_vec_dict={}
for word in vocab:
  word_vec_dict[word]=w2v.wv.get_vector(word)
print(len(word_vec_dict))

35482


In [ ]:
# maxi=-1
# for i,rev in enumerate(df['text']):
#     tokens=rev.split()
#     if(len(tokens)>maxi):
#         maxi=len(tokens)
# print(maxi)

In [ ]:
tok = Tokenizer()
tok.fit_on_texts(df['text'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['text'])

In [ ]:
max_rev_len=125  # max lenght of a review
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim=100

In [ ]:
from keras.preprocessing.sequence import pad_sequences
pad_rev= pad_sequences(encd_rev, maxlen=max_rev_len, padding='post')
pad_rev.shape 

(24734, 125)

In [ ]:
pad_rev

array([[ 44, 524,   1, ...,   0,   0,   0],
       [ 94,  11,  77, ...,   0,   0,   0],
       [ 14,  29,  19, ...,  22,  20,  59],
       ...,
       [ 39, 218,  42, ...,   0,   0,   0],
       [  7, 270,  70, ...,   0,   0,   0],
       [ 92,   1,  19, ...,   0,   0,   0]], dtype=int32)

In [ ]:
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
  embed_vector=word_vec_dict.get(word)
  if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

In [ ]:
embed_matrix.shape

(18184, 100)

In [ ]:
import keras
import keras.utils
from keras import utils as np_utils
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.utils import np_utils
sklearn.model_selection.train_test_split

<function sklearn.model_selection._split.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)>

In [ ]:
# !pip install np_utils
from keras.utils.np_utils import to_categorical

In [ ]:
# Y=keras.utils.to_categorical(df['label'])
# Y.shape
from keras import utils as np_utils
Y=keras.utils.np_utils.to_categorical(df['label'])  # one hot target as required by NN.

In [ ]:
y = np.array(df['label'])

In [ ]:
y = y.reshape(y.size,1)
print(y)

[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [0]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(pad_rev, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.9 x 0.8 = 0.2


In [ ]:
from keras.initializers import Constant
from keras.layers import ReLU
from keras.layers import Dropout
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_dim,input_length=max_rev_len,embeddings_initializer=Constant(embed_matrix)))
model.add(GRU(128))
model.add(Flatten())
model.add(Dense(64,activation='relu',kernel_regularizer='l2'))
model.add(Dropout(0.2))

#model.add(Dense(1024,activation='relu'))
# model.add(Dense(64,activation='relu',kernel_regularizer='l2'))
# model.add(Dropout(0.20))
# model.add(Dense(32,activation='relu',kernel_regularizer='l2'))
# model.add(Dense(32,activation='relu',kernel_regularizer='l2'))
# model.add(Dense(1,activation='relu'))
# model.add(Dense(16,activation='relu'))
# model.add(Dropout(0.20))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 125, 100)          1818400   
                                                                 
 gru_13 (GRU)                (None, 128)               88320     
                                                                 
 flatten_13 (Flatten)        (None, 128)               0         
                                                                 
 dense_26 (Dense)            (None, 64)                8256      
                                                                 
 dropout_13 (Dropout)        (None, 64)                0         
                                                                 
 dense_27 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,915,041
Trainable params: 1,915,041
N

In [ ]:
from tensorflow import keras
from keras import optimizers
import tensorflow_addons as tfa

In [ ]:
from tensorflow_addons.optimizers import weight_decay_optimizers
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0005),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=50
batch_size=128

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.fit(X_train,y_train,epochs=epochs,batch_size=batch_size,validation_data=(X_val,y_val),callbacks=[es, mc])

Epoch 1/50
116/116 [==============================] - ETA: 0s - loss: 1.1971 - accuracy: 0.5555
Epoch 1: val_accuracy improved from -inf to 0.72630, saving model to best_model.h5
116/116 [==============================] - 25s 200ms/step - loss: 1.1971 - accuracy: 0.5555 - val_loss: 0.8823 - val_accuracy: 0.7263
Epoch 2/50
116/116 [==============================] - ETA: 0s - loss: 0.7071 - accuracy: 0.7769
Epoch 2: val_accuracy improved from 0.72630 to 0.79341, saving model to best_model.h5
116/116 [==============================] - 23s 200ms/step - loss: 0.7071 - accuracy: 0.7769 - val_loss: 0.6059 - val_accuracy: 0.7934
Epoch 3/50
116/116 [==============================] - ETA: 0s - loss: 0.5195 - accuracy: 0.8197
Epoch 3: val_accuracy improved from 0.79341 to 0.83060, saving model to best_model.h5
116/116 [==============================] - 47s 411ms/step - loss: 0.5195 - accuracy: 0.8197 - val_loss: 0.4504 - val_accuracy: 0.8306
Epoch 4/50
116/116 [==============================] - E

In [ ]:
modelx = keras.models.load_model('best_model.h5')

In [ ]:
y_pred_one_hot_encoded = (modelx.predict(X_test)> 0.5).astype("float")
#y_pred_test = np.array(tf.argmax(y_pred_one_hot_encoded, axis=1))


In [ ]:
print(classification_report(y_test, y_pred_one_hot_encoded,digits=4))

              precision    recall  f1-score   support

           0     0.9070    0.8744    0.8904      2532
           1     0.8731    0.9060    0.8893      2415

    accuracy                         0.8898      4947
   macro avg     0.8901    0.8902    0.8898      4947
weighted avg     0.8905    0.8898    0.8898      4947



In [ ]:
modelx.evaluate(X_test, y_test, batch_size=64)

78/78 [==============================] - 3s 37ms/step - loss: 0.3198 - accuracy: 0.8898


[0.3198244571685791, 0.8898321986198425]

In [ ]:

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

In [ ]:
def text_preprocess(document):
    # xóa html code
    document = remove_html(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    # tách từ
    document = word_tokenize(document, format="text")
    return document

In [ ]:
# lst_stopword = []
# with open('/content/drive/MyDrive/KhanhMD/Data/vietnamese-stopwords.txt','r') as f:
#     for line in f:
#         for word in line.split():
#            lst_stopword.append(word)  
    
# def remove_stopwords(line):
#     words = []
#     for word in line.strip().split():
#         if word not in lst_stopword:
#             words.append(word)
#     return ' '.join(words)

In [ ]:
document = 'Anh_em được sếp đưa đi ăn cua . Cua của quán này to ngon khỏi chê : ) sốt làm cua đặc_biệt ăn_không ngán . Cua nhiều gạch và thịt , chấm cho món cua sốt ở đây 9/10 các món khác nghe em nhân_sự nói không thuộc sở_trường của quán nên đến đây ăn cua sốt là tốt nhất : )'

document = text_preprocess(document)
document = [document]
print(document)
#tok.fit_on_texts(df['text'])
encd = tok.texts_to_sequences(document)
encd

['anh_em được sếp đưa đi ăn cua cua của quán này to ngon khỏi chê sốt làm cua đặc_biệt ăn_không ngán cua nhiều gạch và thịt chấm cho món cua sốt ở đây 9 10 các món khác nghe em nhân_sự nói không thuộc sở_trường của quán nên đến đây ăn cua sốt là tốt nhất']


[[208,
  246,
  28,
  3021,
  430,
  23,
  1,
  408,
  408,
  54,
  7,
  37,
  254,
  10,
  459,
  487,
  224,
  78,
  408,
  135,
  180,
  1,
  4,
  455,
  408,
  50,
  1570,
  8,
  123,
  377,
  32,
  21,
  408,
  224,
  19,
  20,
  696,
  236,
  67,
  21,
  100,
  219,
  246,
  27,
  168,
  40,
  4,
  1035,
  1038,
  684,
  54,
  7,
  16,
  36,
  20,
  1,
  408,
  224,
  5,
  227,
  97]]

In [ ]:
pad= pad_sequences(encd, maxlen=max_rev_len, padding='post')

In [ ]:
predict_x = modelx.predict(pad)
predict_x

array([[0.9405381]], dtype=float32)